Run on google colab

In [ ]:
!pip install -q malariagen_data bed_reader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 26.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.7/71.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.9/775.9 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.9/25.9 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import malariagen_data
import bed_reader
import dask.array as da
import allel
import numpy as np
import plotly.express as px
import pandas as pd
from dask.diagnostics import ProgressBar
import google.colab
import os

In [ ]:
google.colab.drive.mount("drive")

Mounted at drive


In [ ]:
results_dir = "drive/MyDrive/admixture-funestus"
!mkdir -pv "{results_dir}"

In [ ]:
# do this just to check client location
malariagen_data.Af1()

<MalariaGEN Af1 API client>
Storage URL             : gs://vo_afun_release_master_us_central1/
Data releases available : 1.0, 1.1, 1.2, 1.3, 1.4
Results cache           : None
Cohorts analysis        : 20240515
Site filters analysis   : dt_20200416
Software version        : malariagen_data 15.0.1
Client location         : Oregon, United States (Google Cloud us-west1)
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact support@malariagen.net. For API documentation see 
https://malariagen.github.io/malariagen-data-python/v15.0.1/Af1.html

In [ ]:
malariagen_data.Af1?

In [ ]:
# set up access to funetus data
af1 = malariagen_data.Af1(
    "simplecache::gs://vo_afun_release_master_us_central1",
    simplecache=dict(cache_storage="gcs_cache"),
    site_filters_analysis='sc_20220908',
    release=[1.0,1.2]
)
af1

<MalariaGEN Af1 API client>
Storage URL             : simplecache::gs://vo_afun_release_master_us_central1
Data releases available : 1.0, 1.1, 1.2, 1.3, 1.4
Results cache           : None
Cohorts analysis        : 20240515
Site filters analysis   : sc_20220908
Software version        : malariagen_data 15.0.1
Client location         : Oregon, United States (Google Cloud us-west1)
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact support@malariagen.net. For API documentation see 
https://malariagen.github.io/malariagen-data-python/v15.0.1/Af1.html

In [ ]:
df_samples = af1.sample_metadata(sample_sets = ['1.0','small-2023'])

In [ ]:
af1.snp_calls?

In [ ]:
af1.site_mask_ids

('funestus',)

In [ ]:
df_samples.groupby(['release', 'sample_set', 'country']).size()

release  sample_set                     country                         
1.0      1229-VO-GH-DADZIE-VMF00095     Ghana                                36
         1230-VO-GA-CF-AYALA-VMF00045   Central African Republic             10
                                        Gabon                                40
         1231-VO-MULTI-WONDJI-VMF00043  Benin                                37
                                        Cameroon                             45
                                        Democratic Republic of the Congo     34
                                        Ghana                                31
                                        Malawi                               18
                                        Mozambique                           22
                                        Nigeria                              41
                                        Uganda                               49
                                        Zambia                               43
         1232-VO-KE-OCHOMO-VMF00044     Kenya                                81
         1235-VO-MZ-PAAIJMANS-VMF00094  Mozambique                           76
         1236-VO-TZ-OKUMU-VMF00090      Tanzania                             10
         1240-VO-CD-KOEKEMOER-VMF00099  Democratic Republic of the Congo     43
         1240-VO-MZ-KOEKEMOER-VMF00101  Mozambique                           40
1.2      small-2023                     Burkina Faso                        177
dtype: int64

In [ ]:
df_samples.groupby('admin1_iso').size()

,0
admin1_iso,
BF-01,20
BF-03,91
BF-04,30
BF-06,23
BF-07,13
BJ-AQ,37
CD-HU,43
CD-KN,34
CF-BGF,10


In [ ]:
def get_plink_file_path(
    arm,
    n_snps,
    min_minor_ac,
    thin_offset,
    max_missing_an,
):
    return f"{results_dir}/{arm}.{n_snps}.{min_minor_ac}.{thin_offset}.{max_missing_an}"

In [ ]:
def snp_calls_to_plink(
    sample_sets,
    arm,
    region,
    n_snps,
    min_minor_ac,
    thin_offset,
    max_missing_an=0,
):

    plink_file_path = get_plink_file_path(
        arm=arm,
        n_snps=n_snps,
        thin_offset=thin_offset,
        min_minor_ac=min_minor_ac,
        max_missing_an=max_missing_an,
    )
    bed_file_path = f"{plink_file_path}.bed"
    if os.path.exists(bed_file_path):
        return plink_file_path

    print(f">>> building {plink_file_path}")

    print("access SNP calls")
    ds_snps = af1.snp_calls(
        region=region,
        sample_sets=sample_sets,
        site_mask='funestus',
    )

    print("count alleles")
    gt = allel.GenotypeDaskArray(ds_snps["call_genotype"].data)
    with ProgressBar():
        ac = gt.count_alleles(max_allele=3).compute()

    print("ascertain segregating biallelic sites")
    n_chroms = ds_snps.dims["samples"] * 2
    an_called = ac.sum(axis=1)
    an_missing = n_chroms - an_called
    min_ref_ac = min_minor_ac
    max_ref_ac = n_chroms - min_minor_ac
    # here we choose biallelic sites involving the reference allele
    loc_sites = (
        ac.is_biallelic()
        & (ac[:, 0] >= min_ref_ac)
        & (ac[:, 0] <= max_ref_ac)
        & (an_missing <= max_missing_an)
    )
    print(f"ascertained {np.count_nonzero(loc_sites):,} sites")

    print("thin sites")
    ix_sites = np.nonzero(loc_sites)[0]
    thin_step = max(ix_sites.shape[0] // n_snps, 1)
    ix_sites_thinned = ix_sites[thin_offset::thin_step]
    print(f"thinned to {np.count_nonzero(ix_sites_thinned):,} sites")

    print("set up dataset")
    #here isel doesn't matter, because missing data is encoded as -1,-1 genotype
    #but the allele at pos 1 is not necessary the alternate allele at a biallelic site
    ds_snps_asc = (
        ds_snps
        [["variant_contig", "variant_position", "variant_allele", "sample_id", "call_genotype"]]
        .isel(alleles=slice(0, 2))
        .sel(variants=ix_sites_thinned)
    )

    print("compute genotype ref counts")
    gt = ds_snps_asc["call_genotype"].data
    gn_ref = allel.GenotypeDaskArray(gt).to_n_ref(fill=-127)
    with ProgressBar():
        gn_ref = gn_ref.compute()

    print("ensure genotypes vary")
    loc_var = np.any(gn_ref != gn_ref[:, 0, np.newaxis], axis=1)
    print(f"final no. variants {np.count_nonzero(loc_var)}")

    print("load final data")
    with ProgressBar():
        ds_snps_final = (
            ds_snps_asc
            [["variant_contig", "variant_position", "variant_allele", "sample_id"]]
            .isel(variants=loc_var)
        )
    gn_ref_final = gn_ref[loc_var]
    val = gn_ref_final.T
    alleles = ds_snps_final["variant_allele"].values
    properties = {
        "iid": ds_snps_final["sample_id"].values,
        "chromosome": ds_snps_final["variant_contig"].values,
        "bp_position": ds_snps_final["variant_position"].values,
        "allele_1": alleles[:, 0],
        "allele_2": alleles[:, 1],
    }

    print("write plink files")
    bed_reader.to_bed(
        filepath=bed_file_path,
        val=val,
        properties=properties,
        count_A1=True,
    )
    return plink_file_path

In [ ]:
!ls -lh {results_dir}

total 278M
drwx------ 2 root root 4.0K Feb 27 08:47  2L.100000.17.0.17.admixture
-rw------- 1 root root  21M Feb 27 08:47  2L.100000.17.0.17.bed
-rw------- 1 root root 2.8M Feb 27 08:47  2L.100000.17.0.17.bim
-rw------- 1 root root  17K Feb 27 08:47  2L.100000.17.0.17.fam
drwx------ 2 root root 4.0K Mar  3 07:28  2L.100000.17.0.2.admixture
-rw------- 1 root root  22M Mar  3 07:28  2L.100000.17.0.2.bed
-rw------- 1 root root 2.9M Mar  3 07:28  2L.100000.17.0.2.bim
-rw------- 1 root root  17K Mar  3 07:28  2L.100000.17.0.2.fam
drwx------ 2 root root 4.0K Feb 27 11:57  2L.100000.17.1.17.admixture
-rw------- 1 root root  21M Feb 27 11:57  2L.100000.17.1.17.bed
-rw------- 1 root root 2.8M Feb 27 11:57  2L.100000.17.1.17.bim
-rw------- 1 root root  17K Feb 27 11:57  2L.100000.17.1.17.fam
drwx------ 2 root root 4.0K Mar  3 09:43  2L.100000.34.0.17.admixture
-rw------- 1 root root  22M Mar  3 09:43  2L.100000.34.0.17.bed
-rw------- 1 root root 2.9M Mar  3 09:43  2L.100000.34.0.17.bim
-rw------

In [ ]:
!du -hs {results_dir}

737M	drive/MyDrive/admixture-funestus


In [ ]:
!du -hs /tmp/gcs_cache

du: cannot access '/tmp/gcs_cache': No such file or directory


In [ ]:
!wget --no-clobber https://dalexander.github.io/admixture/binaries/admixture_linux-1.3.0.tar.gz

--2025-03-05 09:04:00--  https://dalexander.github.io/admixture/binaries/admixture_linux-1.3.0.tar.gz
Resolving dalexander.github.io (dalexander.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to dalexander.github.io (dalexander.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1916705 (1.8M) [application/gzip]
Saving to: ‘admixture_linux-1.3.0.tar.gz’

admixture_linux-1.3 100%[===================>]   1.83M  --.-KB/s    in 0.05s   

2025-03-05 09:04:01 (35.5 MB/s) - ‘admixture_linux-1.3.0.tar.gz’ saved [1916705/1916705]



In [ ]:
!tar zxvf admixture_linux-1.3.0.tar.gz

dist/admixture_linux-1.3.0/
dist/admixture_linux-1.3.0/README.32.txt
dist/admixture_linux-1.3.0/admixture
dist/admixture_linux-1.3.0/admixture32
dist/admixture_linux-1.3.0/admixture-manual.pdf


In [ ]:
!ln -s dist/admixture_linux-1.3.0/admixture

In [ ]:
!./admixture

****                   ADMIXTURE Version 1.3.0                  ****
****                    Copyright 2008-2015                     ****
****           David Alexander, Suyash Shringarpure,            ****
****                John  Novembre, Ken Lange                   ****
****                                                            ****
****                 Please cite our paper!                     ****
****   Information at www.genetics.ucla.edu/software/admixture  ****

Usage: admixture <input file> <K>
See --help or manual for more advanced usage.


In [ ]:
!./admixture --help

****                   ADMIXTURE Version 1.3.0                  ****
****                    Copyright 2008-2015                     ****
****           David Alexander, Suyash Shringarpure,            ****
****                John  Novembre, Ken Lange                   ****
****                                                            ****
****                 Please cite our paper!                     ****
****   Information at www.genetics.ucla.edu/software/admixture  ****

                                                                              
  ADMIXTURE basic usage:  (see manual for complete reference)                 
    % admixture [options] inputFile K                                         
                                                                              
  where:                                                                      
    K is the number of populations; and                                       
    inputFile may be:                     

In [ ]:
def run_admixture(
    sample_sets,
    arm,
    region,
    n_snps,
    thin_offset,
    min_minor_ac,
    max_missing_an,
    K,
    seed=42,
):

    plink_file_path = snp_calls_to_plink(
        sample_sets=sample_sets, arm=arm, region=region, n_snps=n_snps,
        min_minor_ac=min_minor_ac, thin_offset=thin_offset, max_missing_an=max_missing_an
    )

    bed_file_path = f"{plink_file_path}.bed"
    fam_file_path = f"{plink_file_path}.fam"
    admixture_dir = f"{plink_file_path}.admixture/{seed}"
    !mkdir -pv {admixture_dir}
    Q_file_path = f"{admixture_dir}/{K}.Q"
    P_file_path = f"{admixture_dir}/{K}.P"
    log_file_path = f"{admixture_dir}/{K}.log"

    # run admixture if needed
    if not os.path.exists(Q_file_path):
        print(f"building {admixture_dir} K={K}")
        !./admixture -j2 --cv --seed={seed} {bed_file_path} {K} > {log_file_path}
        # move files to correct location
        stem = plink_file_path.split("/")[-1]
        !mv -v {stem}.{K}.Q {Q_file_path}
        !mv -v {stem}.{K}.P {P_file_path}

    # load results - sample IDs
    df_fam = pd.read_csv(
        fam_file_path,
        sep=" ",
        header=None,
        names=["family_id", "sample_id", "father", "mother", "sex", "phenotype"],
        index_col=False,
    )
    samples = df_fam["sample_id"]
    df_samples = af1.sample_metadata(sample_sets=sample_sets)
    df_samples = (
        df_samples
        .set_index("sample_id")
        .loc[samples]
    )

    # load results - ancestry fractions
    df_q = pd.read_csv(
        Q_file_path,
        sep=" ",
        header=None,
        names=[f"pop{i}" for i in range(K)],
        index_col=False,
    )
    df_q["popmax"] = df_q.idxmax(axis="columns")
    df_q["popmax_frac"] = df_q.apply(lambda row: row[row["popmax"]], axis="columns")
    df_q.set_index(samples, inplace=True)

    df_out = df_q.join(df_samples).reset_index()
    df_out.attrs["K"] = K
    return df_out

In [ ]:
for thin_offset in [0,1,2]:
  for seed in [42,64000,2323]:
    for i in range(2,11):
      run_admixture(
                  sample_sets=['1.0', 'small-2023'],
                  arm='2L',
                  region='2RL:57,604,655-102,000,000',
                  n_snps=50_000,
                  thin_offset=thin_offset,
                  min_minor_ac=17,
                  max_missing_an=2,
                  K=i,
                  seed=seed)

building drive/MyDrive/admixture-funestus/2L.50000.17.2.2.admixture/2323 K=7
copied '2L.50000.17.2.2.7.Q' -> 'drive/MyDrive/admixture-funestus/2L.50000.17.2.2.admixture/2323/7.Q'
removed '2L.50000.17.2.2.7.Q'
copied '2L.50000.17.2.2.7.P' -> 'drive/MyDrive/admixture-funestus/2L.50000.17.2.2.admixture/2323/7.P'
removed '2L.50000.17.2.2.7.P'
building drive/MyDrive/admixture-funestus/2L.50000.17.2.2.admixture/2323 K=8
copied '2L.50000.17.2.2.8.Q' -> 'drive/MyDrive/admixture-funestus/2L.50000.17.2.2.admixture/2323/8.Q'
removed '2L.50000.17.2.2.8.Q'
copied '2L.50000.17.2.2.8.P' -> 'drive/MyDrive/admixture-funestus/2L.50000.17.2.2.admixture/2323/8.P'
removed '2L.50000.17.2.2.8.P'
building drive/MyDrive/admixture-funestus/2L.50000.17.2.2.admixture/2323 K=9
copied '2L.50000.17.2.2.9.Q' -> 'drive/MyDrive/admixture-funestus/2L.50000.17.2.2.admixture/2323/9.Q'
removed '2L.50000.17.2.2.9.Q'
copied '2L.50000.17.2.2.9.P' -> 'drive/MyDrive/admixture-funestus/2L.50000.17.2.2.admixture/2323/9.P'
removed